<a href="https://colab.research.google.com/github/bhumikajunghare/App/blob/main/App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio gTTS requests textblob deep-translator

In [2]:
# Importing Required Libraries
import gradio as gr
from gtts import gTTS
import requests
from textblob import TextBlob
from deep_translator import GoogleTranslator
import os

In [3]:
# Using News API
NEWS_API_URL = "https://newsapi.org/v2/everything"
NEWS_API_KEY = "a2c8427597a64a0fafd9a61c4230e1d4"

In [4]:
# Here we are Defining News API requests
def fetch_news(company_name):
    params = {
        "q": company_name,
        "apiKey": NEWS_API_KEY,
        "pageSize": 3,
        "language": "en"
    }
    # Sending get rquest to news api
    response = requests.get(NEWS_API_URL, params=params)

    # if request successful
    if response.status_code == 200:

      # returing the list of news articles from the API response
        return response.json().get("articles", [])
    else:
        return []

In [5]:
# For Analyzing sentiments
def analyze_sentiment(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"

In [6]:
# for Translating speech into hindi
def translate_to_hindi(text):
    try:
        return GoogleTranslator(source="auto", target="hi").translate(text)
    except Exception as e:
        return f"Translation failed: {str(e)}"

In [7]:
def text_to_speech(text):
    tts = gTTS(text=text, lang="hi", slow=False)
    audio_path = "output.mp3"
    tts.save(audio_path)
    return audio_path

In [8]:
# fetching the news articles related to the given company
def process_news(company_name):
    articles = fetch_news(company_name)
    results = []

    # Extracting the description of the article, analyzing sentiments, translating content and converting translation into Hindi
    for article in articles:
        content = article.get("description", "")
        sentiment = analyze_sentiment(content)
        hindi_text = translate_to_hindi(content)
        audio = text_to_speech(hindi_text)
        results.append((content, sentiment, hindi_text, audio))
    return results

In [10]:
def gradio_interface(company_name):
    results = process_news(company_name)
    output = ""
    audio_paths = []
    for i, (content, sentiment, hindi_text, audio_path) in enumerate(results):
        output += f"News {i+1}:\nEnglish: {content}\nHindi: {hindi_text}\nSentiment: {sentiment}\n\n"
        audio_paths.append(audio_path)
    return output, audio_paths[0] if audio_paths else None

# Creating gradio interface to interact with model
demo = gr.Interface(fn=gradio_interface,
                    inputs="text",
                    outputs=["text", "audio"],
                    title="News Summarization & Sentiment Analysis with Hindi Audio")

# launching gradio web application
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d64855efceee81af03.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
